In [1]:
import warnings
warnings.filterwarnings("ignore")

from time import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd

from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_predict

pd.set_option('display.max_rows', None)

In [2]:
df_crisis_boe = pd.read_excel('/content/sample_data/Bank_ready.xlsx')

df_crisis_mean = pd.read_excel('/content/sample_data/Mean_ready.xlsx') 

df_crisis_median = pd.read_excel('/content/sample_data/Median_ready.xlsx') 

df_crisis_mostf = pd.read_excel('/content/sample_data/Most_f_ready.xlsx')  

In [3]:
def datapreparation(df_crisis):

  dfx_crisis = df_crisis.drop(['year','iso'],axis=1) #retirar atributos sem relação a crises
  
  dfx_crisis = dfx_crisis.drop(['crisis'],axis=1) #retirar variavel binaria indicadora de crises

  dfy_crisis = df_crisis['crisis'].copy() 
  dfx_crisis.shape, dfy_crisis.shape
  
  dfx_crisis_resampled, dfy_crisis_resampled = SMOTE().fit_resample(dfx_crisis, dfy_crisis)
  dfx_crisis_resampled,dfy_crisis_resampled = shuffle(dfx_crisis_resampled,dfy_crisis_resampled)

  dfx_crisis_resampled = pd.DataFrame(dfx_crisis_resampled)
  dfy_crisis_resampled = pd.DataFrame(dfy_crisis_resampled)
  dfx_crisis_resampled.shape, dfy_crisis_resampled.shape

  return dfx_crisis_resampled, dfy_crisis_resampled

In [4]:
dfx_boe, dfy_boe = datapreparation(df_crisis_boe)
dfx_mean, dfy_mean = datapreparation(df_crisis_mean)
dfx_median, dfy_median = datapreparation(df_crisis_median)
dfx_mostf, dfy_mostf = datapreparation(df_crisis_mostf)

In [5]:
dfx_boe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308 entries, 0 to 2307
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2308 non-null   float64
 1   1       2308 non-null   float64
 2   2       2308 non-null   float64
 3   3       2308 non-null   float64
 4   4       2308 non-null   float64
 5   5       2308 non-null   float64
 6   6       2308 non-null   float64
 7   7       2308 non-null   float64
 8   8       2308 non-null   float64
 9   9       2308 non-null   float64
 10  10      2308 non-null   float64
 11  11      2308 non-null   float64
dtypes: float64(12)
memory usage: 216.5 KB


In [6]:
dfy_boe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308 entries, 0 to 2307
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       2308 non-null   int64
dtypes: int64(1)
memory usage: 18.2 KB


In [7]:
# Split the dataset in two equal parts
def spliter(dfx,dfy):
  X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.75, random_state=0)
  return X_train, X_test, y_train, y_test

In [8]:
# Set the parameters by cross-validation
tuned_parameters = [{'criterion': ['gini','entropy'],
                     'max_depth': [None,5,10,13,15,17,20,23,25],
                     'max_features': ['auto','sqrt','log2'],
                     'min_samples_split': [1,2,3,4,5,6,7,8],
                     'min_samples_split': [2,5,10],
                     'n_estimators': [100, 200, 300, 500, 750,1000,1250,1300, 1400, 
                                      1450, 1500 1525, 1550, 1575, 1600 ,1700, 1800,2000]}]

scores = ['precision', 'recall']

SyntaxError: ignored

In [ ]:
def p_estimation(dfx,dfy):
  X_train, X_test, y_train, y_test = spliter(dfx,dfy)
  for score in scores:
      print("# Tuning hyper-parameters for %s" % score)
      print()

      rf = GridSearchCV(
          RandomForestClassifier(), tuned_parameters, scoring='%s_macro' % score
      )
      rf.fit(X_train, y_train)

      print("Best parameters set found on development set:")
      print()
      print(rf.best_params_)
      print()
      print("Grid scores on development set:")
      print()
      means = rf.cv_results_['mean_test_score']
      stds = rf.cv_results_['std_test_score']
      for mean, std, params in zip(means, stds, rf.cv_results_['params']):
          print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params))
      print()

      print("Detailed classification report:")
      print()
      print("The model is trained on the full development set.")
      print("The scores are computed on the full evaluation set.")
      print()
      y_true, y_pred = y_test, rf.predict(X_test)
      print(classification_report(y_true, y_pred))
      print()

In [ ]:
p_estimation(dfx_boe, dfy_boe)

In [ ]:
#p_estimation(dfx_mean, dfy_mean)

In [ ]:
#p_estimation(dfx_median, dfy_median)

In [ ]:
#p_estimation(dfx_mostf, dfy_mostf)